In [1]:
import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor



print("MLFLOW_TRACKING_URI =", os.environ["MLFLOW_TRACKING_URI"])

MLFLOW_TRACKING_URI = http://mlflow-service:5000


In [2]:
diabetes = datasets.load_diabetes(as_frame=True)
df = diabetes.frame.copy()

TARGET = "target"
FEATURES = [c for c in df.columns if c != TARGET]

X = df[FEATURES]
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

df.head()


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [3]:
EXPERIMENT_NAME = "Test"

# Установим эксперимент (создаст, если не существует)
experiment = mlflow.set_experiment(EXPERIMENT_NAME)
print("Experiment:", experiment)

# Проверка через клиента
client = MlflowClient()
existing = client.get_experiment_by_name(EXPERIMENT_NAME)
print("Existing:", existing)


Experiment: <Experiment: artifact_location='s3://mlops-2025/mlflow/385851095250271093', creation_time=1758725373719, experiment_id='385851095250271093', last_update_time=1758725373719, lifecycle_stage='active', name='Test', tags={}>
Existing: <Experiment: artifact_location='s3://mlops-2025/mlflow/385851095250271093', creation_time=1758725373719, experiment_id='385851095250271093', last_update_time=1758725373719, lifecycle_stage='active', name='Test', tags={}>


In [4]:
# Run
with mlflow.start_run(run_name="Simple_run") as run:
    mlflow.set_tag("author", "liza")
    mlflow.log_param("random_state", 42)

    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train)
    y_pred = lr.predict(X_test_scaled)

    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    mae  = float(median_absolute_error(y_test, y_pred))
    r2   = float(r2_score(y_test, y_pred))

    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    print("Run ID:", run.info.run_id, "| RMSE:", rmse)


2025/10/01 16:20:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run Simple_run at: http://mlflow-service:5000/#/experiments/385851095250271093/runs/2565ce0c21f146aa951c5441b8ce8db9.
2025/10/01 16:20:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/385851095250271093.


Run ID: 2565ce0c21f146aa951c5441b8ce8db9 | RMSE: 53.85344583676594


In [5]:
# Родительский ран + дочерний (nested run) с infer_signature
with mlflow.start_run(run_name="Parent") as parent:
    # Общий артефакт
    sample = X.head(10)
    sample_path = "sample.csv"
    sample.to_csv(sample_path, index=False)
    mlflow.log_artifact(str(sample_path), artifact_path="experiment_assets")

    # Дочерний ран
    with mlflow.start_run(run_name="Child_HistGB", nested=True) as child:
        hgb = HistGradientBoostingRegressor(random_state=42)
        hgb.fit(X_train_scaled, y_train)
        y_pred = hgb.predict(X_test_scaled)

        metrics = {
            "rmse": float(np.sqrt(mean_squared_error(y_test, y_pred))),
            "mae":  float(median_absolute_error(y_test, y_pred)),
            "r2":   float(r2_score(y_test, y_pred))
        }
        mlflow.log_metrics(metrics)

        signature = infer_signature(X_test_scaled, y_pred)
        mlflow.sklearn.log_model(hgb, "model", signature=signature, input_example=X_test_scaled[:2])

        print("Child run:", child.info.run_id)

    print("Parent run:", parent.info.run_id)


2025/10/01 16:20:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run Child_HistGB at: http://mlflow-service:5000/#/experiments/385851095250271093/runs/1a1a9d2e7cc64f8b8684787cd79bf4db.
2025/10/01 16:20:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/385851095250271093.
2025/10/01 16:20:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent at: http://mlflow-service:5000/#/experiments/385851095250271093/runs/78bb16fc10b8447ab48701356d9e567c.
2025/10/01 16:20:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/385851095250271093.


Child run: 1a1a9d2e7cc64f8b8684787cd79bf4db
Parent run: 78bb16fc10b8447ab48701356d9e567c


In [10]:
# Работа с Model Registry
REGISTERED_NAME = "DiabetesRegressorDemo"

# 1. Проверяем, есть ли зарегистрированная модель
try:
    client.get_registered_model(REGISTERED_NAME)
    print(f"Модель {REGISTERED_NAME} уже существует")
except Exception:
    client.create_registered_model(REGISTERED_NAME)
    print(f"Создана зарегистрированная модель {REGISTERED_NAME}")


# 2. Добавляем новую версию
run_id = best_uri.split("/")[1]  # runs:/<RUN_ID>/model
mv = client.create_model_version(
    name=REGISTERED_NAME,
    source=mlflow.get_artifact_uri("model"),  # путь внутри рана
    run_id=run_id,
)
print("Создана версия модели:", mv.version)

2025/10/01 16:23:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DiabetesRegressorDemo, version 3


Модель DiabetesRegressorDemo уже существует
Создана версия модели: 3


In [14]:
# 3. Переводим в Staging
client.transition_model_version_stage(
    name=REGISTERED_NAME,
    version=mv.version,
    stage="Staging",
    archive_existing_versions=False
)
print("Версия", mv.version, "переведена в Staging")

# 4. Переводим в Production
client.transition_model_version_stage(
    name=REGISTERED_NAME,
    version=mv.version,
    stage="Production",
    archive_existing_versions=True
)
print("Версия", mv.version, "переведена в Production")

Версия 3 переведена в Staging
Версия 3 переведена в Production


/tmp/ipykernel_237/1979648783.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/tmp/ipykernel_237/1979648783.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
